In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


class ObesityDataset(Dataset):
    def __init__(self, csv_file, train=True, transform=None, test_size=0.2, random_state=42):
        # Load the data
        self.data = pd.read_csv(csv_file)

        # Separate features and target
        self.features = self.data.drop(columns=['NObeyesdad'])
        self.target = self.data['NObeyesdad']

        # Label encode the target
        self.label_encoder = LabelEncoder()
        self.target = self.label_encoder.fit_transform(self.target)

        # Split into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
self.features, self.target, test_size=test_size, random_state=random_state
        )

        # Choose train or test split
        if train:
            self.features, self.target = X_train, y_train
        else:
            self.features, self.target = X_test, y_test

        # Apply transformations
        self.transform = transform
        self._preprocess_features()

    def _preprocess_features(self):
        # Identify categorical and numerical columns
        categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC',
                            'MTRANS']
        numerical_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

        # Encode categorical columns
        self.features[categorical_cols] = self.features[categorical_cols].apply(LabelEncoder().fit_transform)

        # Scale numerical columns
        scaler = StandardScaler()
        self.features[numerical_cols] = scaler.fit_transform(self.features[numerical_cols])

        # Convert to tensor
        self.features = torch.tensor(self.features.values, dtype=torch.float32)
        self.target = torch.tensor(self.target, dtype=torch.long)

    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        # Retrieve a single sample
        sample = self.features[idx]
        target = self.target[idx]

        # Apply any transformations
        if self.transform:
            sample = self.transform(sample)

        return sample, target




In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
 # Import the custom dataset

# Define the neural network model
class ObesityClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(ObesityClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define function to calculate accuracy
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_true).float().sum()
    return (correct / y_true.shape[0]) * 100

# Set hyperparameters
input_size = 16  # Number of features in the dataset
hidden_sizes = [64, 32]  # Hidden layer sizes
output_size = len(set(ObesityDataset(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv").target))  # Number of classes
num_epochs = 100
batch_size = 32
learning_rate = 0.001

# Load dataset and create DataLoader
train_dataset = ObesityDataset(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv", train=True)
test_dataset = ObesityDataset(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv", train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, loss function, and optimizer
model = ObesityClassifier(input_size, hidden_sizes, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()


    # Calculate and display accuracy every 10 epochs
    if (epoch + 1) % 10 == 0:
        model.eval()
        total_correct = 0
        total_samples = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                total_correct += calculate_accuracy(outputs, labels).item() * labels.size(0)
                total_samples += labels.size(0)
        
        accuracy = total_correct / total_samples
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')

trained = torch.save(model, r"C:\Users\arali\Desktop\trained_model.pth")
    


Epoch [10/100], Loss: 0.6904, Accuracy: 80.85%
Epoch [20/100], Loss: 0.3876, Accuracy: 89.60%
Epoch [30/100], Loss: 0.2831, Accuracy: 91.96%
Epoch [40/100], Loss: 0.2184, Accuracy: 89.83%
Epoch [50/100], Loss: 0.1869, Accuracy: 92.20%
Epoch [60/100], Loss: 0.1683, Accuracy: 93.38%
Epoch [70/100], Loss: 0.1469, Accuracy: 95.04%
Epoch [80/100], Loss: 0.1189, Accuracy: 94.56%
Epoch [90/100], Loss: 0.1219, Accuracy: 95.98%
Epoch [100/100], Loss: 0.1059, Accuracy: 94.56%


In [3]:
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler




# Load dataset to fit encoders and scaler based on training data
dataset = pd.read_csv(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv")
categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
numerical_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']

# Label encoding for categorical columns
label_encoders = {col: LabelEncoder().fit(dataset[col]) for col in categorical_cols}
# Standardization for numerical columns
scaler = StandardScaler().fit(dataset[numerical_cols])

# Load the trained model
input_size = 16  # Number of features
hidden_sizes = [64, 32]
output_size = len(set(ObesityDataset(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv").target))  # Number of classes
model = ObesityClassifier(input_size, hidden_sizes, output_size)
model = torch.load(r"C:\Users\arali\Desktop\trained_model.pth")  # Load the model (make sure to save it after training)
model.eval()

# Function to preprocess user input
def preprocess_input(user_input):
    # Convert user input into a DataFrame
    user_df = pd.DataFrame([user_input], columns=categorical_cols + numerical_cols)
    
    # Encode categorical columns
    for col in categorical_cols:
        user_df[col] = label_encoders[col].transform(user_df[col])
    
    # Scale numerical columns
    user_df[numerical_cols] = scaler.transform(user_df[numerical_cols])
    
    # Convert to tensor
    return torch.tensor(user_df.values, dtype=torch.float32)

# Dictionary to prompt the user for input




# Predict function
def predict_obesity_level():
    # Get user input
    user_input1 = {
    'Gender': 'Female',
    'family_history_with_overweight': 'yes',
    'FAVC': 'yes',
    'FCVC': 2.0,
    'NCP': 3.0,
    'CAEC': 'no',
    'SMOKE': 'yes',
    'CH2O': 2.0,
    'SCC': 'yes',
    'FAF': 2.0,
    'TUE': 1.0,
    'CALC': 'Always',
    'MTRANS': 'Public_Transportation',
    'Age': 66.0,
    'Height': 100.0,
    'Weight': 5000.0
}
    
    user_input2 = {
    'Gender': 'Female',
    'family_history_with_overweight': 'no',
    'FAVC': 'yes',
    'FCVC': 3.0,
    'NCP': 2.0,
    'CAEC': 'yes',
    'SMOKE': 'yes',
    'CH2O': 1.5,
    'SCC': 'no',
    'FAF': 0.0,
    'TUE': 2.0,
    'CALC': 'Always',
    'MTRANS': 'Walking',
    'Age': 34.0,
    'Height': 200.0,
    'Weight': 10.0
}
    
    # Preprocess the input
    input_tensor = preprocess_input(user_input1)
    
    # Predict
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted_class = torch.max(output, 1)
    
    # Decode the predicted class
    obesity_classes = ObesityDataset(r"C:\Users\arali\Desktop\ObesityDataSet_raw_and_data_sinthetic.csv").label_encoder.classes_
    prediction = obesity_classes[predicted_class.item()]
    
    print(f"\nPredicted Obesity Level: {prediction}")

    
predict_obesity_level()


Predicted Obesity Level: Obesity_Type_I
